In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D,AlphaDropout,SeparableConv2D, SpatialDropout2D, MaxPooling2D
from tensorflow.keras import regularizers
from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
%matplotlib inline

Using TensorFlow backend.


In [ ]:
!pip install git+https://github.com/rcmalli/keras-vggface.git -q

In [ ]:
from keras_vggface.vggface import VGGFace
sen = VGGFace(model='senet50',include_top=False, input_shape=(200, 200, 3))

104947712/104944616 [==============================] - 4s 0us/step


In [ ]:
fx_train_sen = sen.predict(x_train)
fx_val_sen = sen.predict(x_val)
fx_test_sen = sen.predict(x_test)

In [ ]:
#Loading Labels
traindf = pd.read_excel('/content/drive/My Drive/Facial Demographics/traindf.xlsx')
valdf = pd.read_excel('/content/drive/My Drive/Facial Demographics/valdf.xlsx')
testdf = pd.read_excel('/content/drive/My Drive/Facial Demographics/testdf.xlsx')
traindf.head()

Age Estimation Model

In [ ]:
print('Train : '+str(fx_train_sen.shape))
print('Val : '+str(fx_val_sen.shape))
print('Test : '+str(fx_test_sen.shape))

In [ ]:
model = Sequential()

model.add(Flatten())
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(512,activation='elu'))


model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(512,activation='elu'))

model.add(BatchNormalization())
model.add(Dense(1,activation='relu',kernel_initializer='he_uniform'))

In [ ]:
sgd = tf.keras.optimizers.SGD(
    learning_rate=1e-2, momentum=0.9, nesterov=True, name="SGD")

adam = tf.keras.optimizers.Adam(
    learning_rate=1e-3,
    amsgrad=True,)

adamax = tf.keras.optimizers.Adamax(learning_rate=1e-3)

ftrl = tf.keras.optimizers.Ftrl(
    learning_rate=0.01,
    learning_rate_power=-0.5,
    initial_accumulator_value=0.1,
    l1_regularization_strength=0.0,
    l2_regularization_strength=0.0,
    name="Ftrl",
    l2_shrinkage_regularization_strength=0.0,
)

adadelta = tf.keras.optimizers.Adadelta(
    learning_rate=1e-3, rho=0.95, epsilon=1e-08, name="Adadelta"
)

adagrad=tf.keras.optimizers.Adagrad(
    learning_rate=0.001,
    initial_accumulator_value=0,
    epsilon=1e-07,
    name="Adagrad",
)

model.compile(optimizer=adam,loss='mse',metrics=['mae'])

In [ ]:
model.fit(fx_train_sen,traindf['age'].values ,epochs=20, validation_data=(fx_val_sen,valdf['age'].values),batch_size=256)

In [ ]:
model.evaluate(fx_test_sen,testdf['age'].values)

In [ ]:
model.predict(fx_test_vgg).shape

Ensemble

In [ ]:
from keras.models import load_model

G_res = load_model('/content/drive/My Drive/Facial Demographics/Resnet.h5')
G_res.compile("Adam","binary_crossentropy",['acc'])

G_sen = load_model('/content/drive/My Drive/Facial Demographics/Senet.h5')
G_sen.compile("Adam","binary_crossentropy",['acc'])

G_vgg = load_model('/content/drive/My Drive/Facial Demographics/Vgg.h5')
G_vgg.compile("Adam","binary_crossentropy",['acc'])

In [ ]:
fx_test_res_e = np.load('/content/drive/My Drive/Facial Demographics/fx_test_res.npy')
fx_test_sen_e = np.load('/content/drive/My Drive/Facial Demographics/fx_test_sen.npy')
fx_test_vgg_e = np.load('/content/drive/My Drive/Facial Demographics/fx_test_vgg.npy')

In [ ]:
pred_res = A_res.predict(fx_test_res_e)
pred_sen = A_sen.predict(fx_test_sen_e)
pred_vgg = A_vgg.predict(fx_test_vgg_e)

In [ ]:
pred_res.shape, pred_sen.shape, pred_vgg.shape

In [ ]:
y_pred = (pred_res+pred_sen+pred_vgg)/3

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(testdf['gender'].values,y_pred.round())
print(report)

In [ ]:
from sklearn.metrics import roc_auc_score

auc_vgg = roc_auc_score(testdf['gender'].values,pred_vgg.round())
auc_res = roc_auc_score(testdf['gender'].values,pred_res.round())
auc_sen = roc_auc_score(testdf['gender'].values,pred_sen.round())
auc_en = roc_auc_score(testdf['gender'].values,y_pred.round())
print('ROC_AUC_SCORE : \n')
print('ResNet50: ',auc_res)
print('SeNet50: ',auc_sen)
print('VGG16: ',auc_vgg)
print('Ensemble: ',auc_en)

In [ ]:
from sklearn.metrics import accuracy_score 

acc_vgg = accuracy_score(testdf['gender'].values,pred_vgg.round())
acc_res = accuracy_score(testdf['gender'].values,pred_res.round())
acc_sen = accuracy_score(testdf['gender'].values,pred_sen.round())
acc_en = accuracy_score(testdf['gender'].values,y_pred.round())
print('ACCURACY_SCORE : \n')
print('ResNet50: ',acc_res)
print('SeNet50: ',acc_sen)
print('VGG16: ',acc_vgg)
print('Ensemble: ',acc_en)

Age

In [ ]:
testdf['age_pred_res'] = pred_res 
testdf['age_pred_sen'] = pred_sen 
testdf['age_pred_vgg'] = pred_vgg 

In [ ]:
testdf

,Unnamed: 0,age,gender,race,file,age_pred_res,age_pred_sen,age_pred_vgg
0,6140,36,1,4,36_1_4_20170117203519312.jpg.chip.jpg,24.485363,27.753326,28.203520
1,9956,53,1,0,53_1_0_20170110123801468.jpg.chip.jpg,38.145927,37.904057,37.676086
2,1549,26,1,4,26_1_4_20170116220414012.jpg.chip.jpg,24.097799,24.639765,25.128021
3,14818,54,0,3,54_0_3_20170119210127256.jpg.chip.jpg,55.328480,46.717438,44.873314
4,18428,50,0,0,50_0_0_20170111171747253.jpg.chip.jpg,42.199821,41.806408,44.828255
...,...,...,...,...,...,...,...,...
2366,21575,58,0,0,58_0_0_20170117135834808.jpg.chip.jpg,50.158852,48.580410,45.263424
2367,5390,26,1,2,26_1_2_20170116184144892.jpg.chip.jpg,25.099579,28.005995,24.960840
2368,860,57,0,0,57_0_0_20170116214453175.jpg.chip.jpg,39.390530,50.115044,38.975800
2369,15795,55,0,0,55_0_0_20170117155725626.jpg.chip.jpg,54.393230,54.313946,52.882278


In [ ]:

plt.plot(sample['age'],sample['age_pred_res'], alpha=0.5)
plt.plot(sample['age'],sample['age_pred_sen'], alpha=0.5)
plt.plot(sample['age'],sample['age_pred_vgg'], alpha=0.5)
plt.plot(sample['age'],sample['age'])
plt.show()